<a href="https://colab.research.google.com/github/gtrofimiuk/ML_course/blob/HW1/MusicGenres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Датасет https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre/data

Изначально я скачивал через код ниже, но, оказалось, что DATA_SOURCE_MAPPING временный. В принципе, датасет уже есть в репозитории, ну или можно взять из kaggle

В любом случае, папки создам

In [1]:
import os
os.makedirs("sample_data/prediction-of-music-genre", 0o777, exist_ok=True)

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'prediction-of-music-genre:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1690118%2F2769452%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240131%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240131T163614Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da7baa0c32bc50dbc5f8a6acb57083e776b5012d59bae1d3c5933a39ffc813ef54f87ae16c8cdf17e0b7283cf4db66ddbdc8bbdf868d445fb511d710633eedb4fc62278a68f8b3c8eb85b128f44e128f069181ac5c585c014607f2fe9ee77cba72572d05edf6ff87243b28de02b6f261e905d1dd48c088c9a7463cddd55b5005bc5954343682d3511e6393190d67993e328dc027c8a82f1bf3e47cc52239ac5de04488d4bac6a47de135b7a3645af62d0c07639f1b32b0ac039a5cf9d82805c4e6c22f98d967f9cb5aa6d26e2f2af56f63a1301058fe6e540057e95a0ce6c17e030d47265e5f5e073b66760f188375e21ebcbd0707ae64da63257d1c13d5e4a00'


for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join("sample_data", directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [2]:
!pip install basemap > None

In [3]:
!pip install --upgrade matplotlib > None # need refresh enviroment after installation

In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.cbook import boxplot_stats
from mpl_toolkits.basemap import Basemap

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

%matplotlib inline

# Данные

Набор данных состоит из различных параметров музыкальных треков

In [13]:
dataset_path = "sample_data/prediction-of-music-genre/music_genre.csv"
data = pd.read_csv(dataset_path)

In [ ]:
data.head()

In [7]:
data.shape

(50005, 18)

## Признаки в датасете

In [8]:
data.dtypes

instance_id         float64
artist_name          object
track_name           object
popularity          float64
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo                object
obtained_date        object
valence             float64
music_genre          object
dtype: object

Как можно увидеть, изначально у нас есть 18 описание которых можно найти на https://developer.spotify.com/documentation/web-api/reference/get-audio-features

Ниже курсивом буду писать описание из этого сайта

(Отмечу, что на данный момент Spotify запрещает использовать свое API для тренировки моделей машинного обучения, но, в РФ они не работают, да и датасет лежит в открытом доступе, так что ладно. Но вот обновить его уже сложно)

* instance_id - Идентификатор, без особого смысла
* artist_name - Исполнитель песни
* track_name - Название трека
* popularity - *The popularity of the track. The value will be between 0 and 100, with 100 being the most popular.  The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are.
Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past*
* acousticness - *A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic*

* danceability - *Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable*

* duration_ms - *The duration of the track in milliseconds*

* energy - *Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy*

* instrumentalness - *Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0*

* key - *The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.*

  В русской музыкальной традции ближайший аналог - тональность

* liveness - *Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.*

  Сомнительно, что это можно использовать для классификации, но посмотрим. Возможно, что есть жанры, где чаще записывают лайвы

* loudness - *The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db*

  Тоже вопрос, как эта метрика измерялась. Наверное, некоторые жанры такой признак может различить, поскольку электро обычно сильно громче блюза. Но, конечно же, многое зависит от сведения и мастеринга у трека

* mode - *Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0*

  Ближайший аналог - лад, но, лично я никогда не видел, чтобы key и mode рассматривались отдельно

* speechiness -*Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks*

  Тут конечно некоторые жанры можно сильно различить. Например, рэп от классической музыки

* tempo - *The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration*

* obtained_date - дата извлечения из датасета, тоже смысла не имеет

* valence - *A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)*

* music_genre - Жанр музыки. Вообще, если бы это была серьезная работа для индустрии, то надо было бы заморочиться с тем, что мы считаем каким жанром. Тем более, не редки смеси жанров. Но, сейчас, в учебных целях, будем считать что Spotify разметил данные вот так и будем предсказывать по этой модели


**target: music_genre**


In [14]:
df = data.drop('instance_id', axis=1)
df.drop('obtained_date', inplace=True, axis=1)

In [ ]:
df.head()

In [19]:
df.shape

(50005, 16)

## Конструирование признаков

Я все же вижу смысл модели посмотреть на элементарные признаки из текста.

Количество слов у исполнителя,
Наличие скобок в тексте песни


На будущее

Преобразовать тональности по кварто-квинтовому кругу в круг
И по хроматической гамме

# Кодирование категориальных данных



## Тональность

По своей сути, тональность (key) имеет циклическую структру, более того, круги можно сделать даже разные

*   Хроматический круг - берем ноты по порядку: C, C#, D, D#,...
*   Кварто-квинтовый круг для мажора - C, G, D, A, E, B,...
*   Кварто-квинтовый круг для минора - A, E, B, F#, C#

По сути, тут кодирование такое же, как и у времени
https://stats.stackexchange.com/questions/311494/best-practice-for-encoding-datetime-in-machine-learning

## Жанры

Разныз жанров немного, поэтому начнем с OneHot Encoding